## With SentenceTransformer

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('C:/Users/kiran/Python_3/quora-question-pairs_2021/train.csv/train_for_similarity.txt', delimiter = "\t",header=None)
df = df[0:25000]
#df1 = df.iloc[0::2]
#df2 = df.iloc[1::2]
#df1.reset_index(drop=True,inplace=True)
#df2.reset_index(drop=True,inplace=True)

In [114]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [125]:
sentence_embeddings = model.encode(df[0])

In [129]:
sentence_embeddings1 = model.encode("What is the best course for web development?")
from sklearn.metrics.pairwise import cosine_similarity

sim_arr=cosine_similarity(
    [sentence_embeddings1],
    sentence_embeddings[:]
)
di=dict(zip(sim_arr[0],df[0]))

sim_arr=np.sort(sim_arr[0])[::-1]
sim_arr=sim_arr[0:10]

for i in sim_arr:
    print(di[i])

Which language is the best for web development?
What are the best web products to develop?
What are the best productivity tools for web development?
What are the best programming languages to learn for web development?
What is the best Perl web app framework?
What are the best websites to prepare for placements?
What is the best way to get back into web application penetration testing?
What's the best way to find a job as a web developer?
What is your favourite web programming language?
What are good ways to learn how to improve web development?


## Doing it without SentenceTransformer

In [49]:
from transformers import AutoTokenizer, AutoModel
import torch

In [54]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

In [55]:
# initialize dictionary to store tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [56]:
outputs = model(**tokens)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [57]:
embeddings = outputs.last_hidden_state
embeddings

tensor([[[-0.0052, -0.6296,  1.2486,  ..., -0.0733, -0.2491, -0.0506],
         [ 0.1923, -0.9253,  1.3169,  ...,  0.1456,  0.0132, -0.2964],
         [ 0.1014, -0.9976,  1.2731,  ...,  0.1544, -0.1575, -0.0798],
         ...,
         [ 0.3130, -0.7524,  1.1886,  ..., -0.1194, -0.2089,  0.0165],
         [ 0.2546, -0.7417,  1.1883,  ..., -0.1229, -0.2080,  0.0803],
         [ 0.2244, -0.7082,  1.2314,  ..., -0.0896, -0.2728,  0.0759]],

        [[-0.4668, -0.0189,  0.4151,  ...,  0.2630, -0.4392, -0.1010],
         [-0.6711, -0.2192,  0.2173,  ...,  0.0236, -0.2320, -0.1433],
         [-0.5410,  0.3427, -0.0292,  ...,  0.1073, -0.5477,  0.0679],
         ...,
         [-0.3078,  0.0607,  0.2494,  ...,  0.2835, -0.7488, -0.0522],
         [-0.3585,  0.0552,  0.1931,  ...,  0.2523, -0.7587, -0.0163],
         [-0.4766, -0.0714,  0.2329,  ...,  0.2959, -0.6838, -0.1150]],

        [[ 0.0349, -0.6494,  1.6127,  ..., -0.0366, -0.0330, -0.1680],
         [ 0.1675, -0.9080,  1.5879,  ...,  0

In [58]:
embeddings.shape

torch.Size([4, 128, 768])

In [59]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([4, 128])

In [61]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([4, 128, 768])

In [63]:
mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

In [64]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([4, 128, 768])

In [65]:
masked_embeddings

tensor([[[-0.0052, -0.6296,  1.2486,  ..., -0.0733, -0.2491, -0.0506],
         [ 0.1923, -0.9253,  1.3169,  ...,  0.1456,  0.0132, -0.2964],
         [ 0.1014, -0.9976,  1.2731,  ...,  0.1544, -0.1575, -0.0798],
         ...,
         [ 0.0000, -0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [ 0.0000, -0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000],
         [ 0.0000, -0.0000,  0.0000,  ..., -0.0000, -0.0000,  0.0000]],

        [[-0.4668, -0.0189,  0.4151,  ...,  0.2630, -0.4392, -0.1010],
         [-0.6711, -0.2192,  0.2173,  ...,  0.0236, -0.2320, -0.1433],
         [-0.5410,  0.3427, -0.0292,  ...,  0.1073, -0.5477,  0.0679],
         ...,
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
         [-0.0000, -0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000]],

        [[ 0.0349, -0.6494,  1.6127,  ..., -0.0366, -0.0330, -0.1680],
         [ 0.1675, -0.9080,  1.5879,  ...,  0

In [66]:

summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([4, 768])

In [67]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([4, 768])

In [68]:
mean_pooled = summed / summed_mask

In [69]:
mean_pooled

tensor([[-0.0398, -0.8550,  1.2641,  ...,  0.0609, -0.2783, -0.2472],
        [-0.7844,  0.0238,  0.2405,  ...,  0.2124, -0.5366, -0.1705],
        [-0.0281, -0.8078,  1.5523,  ...,  0.0903, -0.0217, -0.3451],
        [ 0.0547, -0.8911,  1.3226,  ...,  0.4829, -0.0389, -0.1081]],
       grad_fn=<DivBackward0>)

In [70]:
# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.44401556, 0.95029956, 0.9565913 ]], dtype=float32)

In [120]:
from numpy import genfromtxt
my_data = genfromtxt('embed.csv', delimiter=',')